In [13]:
from fredapi import Fred
import pandas as pd
import numpy as np

Macro Data Used In Paper
1.	Consumption Growth
- definition：delta_log (Comsumption of goods and service)
- freq: monthly/quarterly

2.	GDP Growth（GDP 增长）
- definition：delta_log(Real GDP)
- freq：quarterly

3.	Industrial Production Growth
- definition: delta_log(Industrial Production Index)
- freq: monthly

4.	Unemployment Rate
- freq: monthly

real-time vintage

In [14]:
FRED_API_KEY="01f9cf5d33262618b310c512b748b70c"

fred = Fred(api_key=FRED_API_KEY)

In [15]:
START = "1986-01-01"
END   = "2019-12-31"

# 1. Real consumption
pce = fred.get_series("PCEC96", observation_start=START, observation_end=END)

# 2. Real GDP
gdp = fred.get_series("GDPC1", observation_start=START, observation_end=END)

# 3. Industrial production
indpro = fred.get_series("INDPRO", observation_start=START, observation_end=END)

# 4. Unemployment rate
unrate = fred.get_series("UNRATE", observation_start=START, observation_end=END)

In [16]:
def log_growth(x, periods=1):
    return np.log(x).diff(periods)

macro = pd.DataFrame({
    "consumption_growth": log_growth(pce),
    "gdp_growth": log_growth(gdp),
    "indpro_growth": log_growth(indpro),
    "unemployment_rate": unrate
})

In [ ]:
macro_monthly = macro.copy()

# 转成月末索引
macro_monthly = macro_monthly.resample("M").last()

# GDP growth forward-fill
macro_monthly["gdp_growth"] = macro["gdp_growth"].resample("M").ffill()

real time vintage

In [17]:
gdp_2008_vintage = fred.get_series(
    "GDPC1",
    observation_start="2005-01-01",
    observation_end="2010-12-31",
    vintage_dates="2008-09-30"
)

In [18]:
vintages = ["2007-12-31", "2008-09-30", "2009-06-30"]

gdp_vintage_panel = pd.concat(
    {
        v: fred.get_series(
            "GDPC1",
            observation_start="2005-01-01",
            observation_end="2010-12-31",
            vintage_dates=v
        )
        for v in vintages
    },
    axis=1
)

gdp_vintage_panel.columns.name = "vintage_date"

In [19]:
gdp_vintage_panel

vintage_date,2007-12-31,2008-09-30,2009-06-30
2005-01-01,10878.4,10875.8,10875.8
2005-04-01,10954.1,10946.1,10946.1
2005-07-01,11074.3,11050.0,11050.0
2005-10-01,11107.2,11086.1,11086.1
2006-01-01,11238.7,11217.3,11217.3
2006-04-01,11306.7,11291.7,11291.7
2006-07-01,11336.7,11314.1,11314.1
2006-10-01,11395.5,11356.4,11356.4
2007-01-01,11412.6,11357.8,11357.8
2007-04-01,11520.1,11491.4,11491.4
